pipe and filter

In [11]:
from abc import ABC, abstractmethod
from typing import List

# Filter Abstract Base Class
class Filter(ABC):
    @abstractmethod
    def process(self, data: dict) -> dict:
        pass

# Concrete Filters (Examples of processing steps)
class SpeedFilter(Filter):
    def process(self, data: dict) -> dict:
        if 'speed' in data:
            data['speed'] = data['speed'] * 1.1  # Increase speed by 10% (for example)
        return data

class RouteFilter(Filter):
    def process(self, data: dict) -> dict:
        if 'route' in data:
            data['route'] = f"Optimized {data['route']}"  # Mock route optimization
        return data

# Pipe to connect filters
class Pipe:
    def __init__(self, filters: List[Filter]):
        self.filters = filters

    def process(self, data: dict) -> dict:
        for filter in self.filters:
            data = filter.process(data)
        return data

# TransportSystem orchestrating the data flow through the Pipe and Filters
class TransportSystem:
    def __init__(self):
        self.pipe = None
    
    def setup(self, filters: List[Filter]):
        self.pipe = Pipe(filters)
    
    def process_data(self, data: dict) -> dict:
        return self.pipe.process(data)


Observer pattern

In [12]:
# Observer Pattern Implementation
# Subject (Publisher)
class Subject(ABC):
    def __init__(self):
        self._observers = []
    
    def add_observer(self, observer):
        self._observers.append(observer)
    
    def remove_observer(self, observer):
        self._observers.remove(observer)
    
    def notify_observers(self):
        for observer in self._observers:
            observer.update(self._state)  # Pass the state to the observer

# Observer (Subscriber)
class Observer(ABC):
    @abstractmethod
    def update(self, state: dict):  # Ensure this method accepts 'state'
        pass

# ConcreteSubject (specific subject being observed)
class TrafficSystem(Subject):
    def __init__(self):
        super().__init__()
        self._state = {}
    
    def set_state(self, state: dict):
        self._state = state
        self.notify_observers()  # Notify observers after state change
    
    def get_state(self):
        return self._state

# ConcreteObserver
class VehicleMonitor(Observer):
    def __init__(self, vehicle_id: str):
        self.vehicle_id = vehicle_id
    
    def update(self, state: dict):  # Accept the state parameter
        print(f"Vehicle {self.vehicle_id} received update: {state}")
        # Here the vehicle could adjust its route, speed, or behavior

class TrafficLight(Observer):
    def update(self, state: dict):  # Accept the state parameter
        print(f"Traffic Light changed: {state}")
        # Example: Change traffic light state based on updated traffic data


LAYER ARCHITECTURE BUSINESS LAYER,PERESENTATION,DATA ACCESS

In [13]:
# Presentation Layer
class PresentationLayer:
    def __init__(self, transport_system: TransportSystem):
        self.transport_system = transport_system
    
    def show_data(self, data: dict):
        print("Displaying processed transport data:")
        print(data)

# Business Logic Layer
class BusinessLogicLayer:
    def __init__(self, transport_system: TransportSystem):
        self.transport_system = transport_system
    
    def calculate_route(self, vehicle_data: dict):
        print(f"Calculating optimized route for vehicle: {vehicle_data['vehicle_id']}")
        vehicle_data['route'] = 'Route A'
        return vehicle_data
    
    def manage_traffic(self, traffic_data: dict):
        print(f"Managing traffic based on data: {traffic_data}")
        traffic_data['status'] = 'Clear'
        return traffic_data

# Data Access Layer (Mock database interaction)
class DataAccessLayer:
    def __init__(self):
        self.data = {}
    
    def store_data(self, data: dict):
        print(f"Storing data: {data}")
        self.data[data['vehicle_id']] = data
    
    def retrieve_data(self, vehicle_id: str):
        return self.data.get(vehicle_id, None)


PUTTING IT TOGETHER

In [14]:
# Main function to demonstrate the public transport system

def main():
    # Setup the filters
    speed_filter = SpeedFilter()
    route_filter = RouteFilter()
    transport_system = TransportSystem()
    transport_system.setup([speed_filter, route_filter])

    # Create vehicle data
    vehicle_data = {
        'vehicle_id': 'V123',
        'speed': 50,  # Initial speed
        'route': 'Route B',  # Initial route
    }
    
    # Create Traffic System and Observers
    traffic_system = TrafficSystem()
    vehicle1 = VehicleMonitor('V123')
    traffic_light = TrafficLight()
    
    traffic_system.add_observer(vehicle1)
    traffic_system.add_observer(traffic_light)

    # Create Layers
    presentation_layer = PresentationLayer(transport_system)
    business_logic_layer = BusinessLogicLayer(transport_system)
    data_access_layer = DataAccessLayer()

    # Business Logic: Calculate Route and Manage Traffic
    updated_vehicle_data = business_logic_layer.calculate_route(vehicle_data)
    updated_traffic_data = business_logic_layer.manage_traffic(updated_vehicle_data)

    # Data Access Layer: Store Vehicle Data
    data_access_layer.store_data(updated_traffic_data)

    # Notify Observers of State Change
    traffic_system.set_state(updated_traffic_data)
    
    # Process the data through filters (Pipe and Filter pattern)
    processed_data = transport_system.process_data(updated_traffic_data)
    
    # Presentation Layer: Show Processed Data
    presentation_layer.show_data(processed_data)

if __name__ == "__main__":
    main()


Calculating optimized route for vehicle: V123
Managing traffic based on data: {'vehicle_id': 'V123', 'speed': 50, 'route': 'Route A'}
Storing data: {'vehicle_id': 'V123', 'speed': 50, 'route': 'Route A', 'status': 'Clear'}
Vehicle V123 received update: {'vehicle_id': 'V123', 'speed': 50, 'route': 'Route A', 'status': 'Clear'}
Traffic Light changed: {'vehicle_id': 'V123', 'speed': 50, 'route': 'Route A', 'status': 'Clear'}
Displaying processed transport data:
{'vehicle_id': 'V123', 'speed': 55.00000000000001, 'route': 'Optimized Route A', 'status': 'Clear'}
